# Import libraries

In [1]:
import json
import re
import multiprocessing
from tqdm import tqdm
from pyvi import ViTokenizer, ViPosTagger
from collections import Counter

# Configure

In [2]:
input_path = '../data/news_dataset_lite.json'
output_data = '../data/processed_data_lite.json'
output_vocab = '../data/vocab_lite.json'

In [8]:
import json

# Đọc dữ liệu từ file JSON
with open('../data/vocab_lite.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Trích xuất trường "content"
# contents = [item['content'] for item in data if 'content' in item]

# Tính tổng số từ trong trường "content"
# total_words = sum(len(content.split()) for content in contents)

# In ra kết quả
print(f'Tổng số từ trong trường "content": {len(data)}')

Tổng số từ trong trường "content": 20821


# Utils

In [3]:
def clean_text(text):
    text = re.sub(r"https?://\S+|www\.\S+", " ", text)  # Loại bỏ URL
    text = re.sub(r"\([^)]*\)", " ", text)  # Loại bỏ nội dung trong ngoặc
    text = re.sub(r"\b\w*\d\w*\b", " ", text)  # Loại bỏ số
    texts = re.split(r'(?<=[.,!?;])\s+', text.strip())  # Tách câu
    texts = [re.sub(r"[^\w\s]", " ", text) for text in texts]  # Loại bỏ ký tự đặc biệt
    return texts

In [4]:
# Hàm xử lý tách từ cho một item
def process_item(item):
    # Tách từ cho từng dòng trong 'contents'
    tokenized_contents = [ViTokenizer.tokenize(it) for it in item['contents']]
    item['contents'] = []

    for tokenized_line in tokenized_contents:
        # Gán nhãn từ loại
        words, pos_tags = ViPosTagger.postagging(tokenized_line)

        # Loại bỏ các từ loại 'Np' (danh từ riêng)
        filtered_words = [word for word, pos in zip(words, pos_tags) if pos != 'Np']
        item['contents'].append(" ".join(filtered_words))  # Gộp lại thành câu

    return item

# Hàm chính để xử lý song song
def parallel_processing(data):
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = list(tqdm(pool.imap(process_item, data), total=len(data)))
    return results

# Main

In [5]:
# Load dataset
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

In [6]:
# Lấy những thông tin cần thiết
filtered_data = [
    {
        "id": item["id"],
        "content": item["content"]
    }
    for item in tqdm(data)
]
data = filtered_data

100%|██████████| 5527/5527 [00:00<00:00, 1692605.01it/s]


In [7]:
# Loại bỏ nội dung không cần thiết
for item in tqdm(data):
    item["contents"] = clean_text(item["content"])
    del item["content"]

100%|██████████| 5527/5527 [00:01<00:00, 4923.69it/s]


In [8]:
data = parallel_processing(data)

100%|██████████| 5527/5527 [00:07<00:00, 746.89it/s]


In [9]:
# Chuyển thành chữ thường
for item in tqdm(data):
    item["contents"] = [it.lower() for it in item["contents"]]

100%|██████████| 5527/5527 [00:00<00:00, 66812.46it/s]


In [10]:
# Xoá các item không có nội dung hoặc nội dung rỗng
data = [item for item in data if item["contents"] and all([it for it in item["contents"]])]

# Gộp tất cả các từ lại
all_words = " ".join([it for item in tqdm(data) for it in item["contents"]]).split()
all_words = [word.strip() for word in all_words]

# Đếm tần suất xuất hiện của các từ
vocab = Counter(all_words)

# Tạo từ điển từ vựng (word2idx)
word_to_idx = {word: idx for idx, word in tqdm(enumerate(vocab.keys()))}

100%|██████████| 2964/2964 [00:00<00:00, 455198.16it/s]
20821it [00:00, 3383163.66it/s]


In [11]:
# Lưu dữ liệu
with open(output_data, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

with open(output_vocab, "w", encoding="utf-8") as f:
    json.dump(word_to_idx, f, ensure_ascii=False, indent=4)